In [ ]:
#Create table
import sqlite3


def insertDB(db_name, data_name, data, date='', filename=''):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    
    if data_name == "cars_raw":
        #print("Cars data filename is ", filename)
        # Create table
        c.execute("""CREATE TABLE IF NOT EXISTS cars_raw (ind integer, link text, name text, price integer, kms integer, year integer, insert_date date, batchname varchar(50))""")
        
        count=conn.execute("select count(*) from cars_raw")
        for cou in count:
            count=cou[0]
        # Larger example that inserts many records at a time
        data_insert=[]
        for d in data:
            #Generate the rows
            count = count+1
            data_insert.append([count, d[0], d[1], d[2], d[3], d[4], date, filename])
        c.executemany('INSERT INTO cars_raw VALUES (?,?,?,?,?,?,?,?)', data_insert)
    
    elif data_name == "cars":
        #print("Cars data filename is ", filename)
        # Create table
        c.execute("""CREATE TABLE IF NOT EXISTS cars (ind integer, link text, name text, price integer, kms integer, year integer, status text, created date, updated date)""")
        
        count=conn.execute("select count(*) from cars")
        for cou in count:
            count=cou[0]
        # Larger example that inserts many records at a time
        data_insert=[]
        for d in data:
            #Generate the rows
            count = count+1
            data_insert.append([count, d[0], d[1], d[2], d[3], d[4], d[5], filename, date, date])
        c.executemany('INSERT INTO cars VALUES (?,?,?,?,?,?,?,?,?,?)', data_insert)

    elif data_name == "test":
        count=conn.execute("select count(*) from "+data_name)
        for cou in count:
            count=cou[0]
        # Larger example that inserts many records at a time
        data_insert=[]
        for d in data:
            #Generate the rows
            count = count+1
            data_insert.append([count, d[0], d[1], d[2], d[3], d[4], date])
        c.executemany('INSERT INTO ' + data_name + ' VALUES (?,?,?,?,?,?,?)', data_insert)

    # Save (commit) the changes
    conn.commit()
    # Close the connection
    conn.close()
        
def createTable(columns, data_name):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    create_string="CREATE TABLE IF NOT EXISTS " + data_name + "(" + columns + ")"
    print(create_string)
    c.execute(create_string)
    conn.commit()
    conn.close()
    
def getCars():
    conn = sqlite3.connect(db_name)
    for row in conn.execute("select * from cars"):
        print (row)
    conn.close()
        
def get(db_name,data_name):
    conn = sqlite3.connect(db_name)
    for row in conn.execute("select * from "+data_name):
        print (row)
    conn.close()
        
def getColumn(data_name, column, db_name):
    conn = sqlite3.connect(db_name)
    l=[]
    for row in conn.execute("select "+column+" from "+data_name):
        l.append (row)
    conn.close()
    return l
    
def deleteCars(db_name):
    conn = sqlite3.connect(db_name)
    conn.execute("delete from cars")
    conn.commit()
    conn.close()
    
def dropCars(db_name):
    conn = sqlite3.connect(db_name)
    conn.execute("drop table cars")
    conn.commit()
    conn.close()
    
def delete(db_name,data_name):
    conn = sqlite3.connect(db_name)
    conn.execute("delete from "+ data_name)
    conn.commit()
    conn.close()


In [ ]:
'''db_name="myown.db"
data_name="test"
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
columns="ind integer, link text, name text, price integer, kms integer, year integer, insert_date date"
#get(data_name)
print("Creating")
createTable(columns, data_name)
get(db_name,data_name)
print("Inserting")
insertDB(db_name, data_name, purchases, "2018-10-23")
print(get(db_name,data_name))
print("Deleting")
delete(db_name,data_name)
get(db_name,data_name)
print("Done")'''

In [ ]:
import os,csv
#Generate database based on csv. Insert matching csv:s with function from first cell.

def getFilesInserted():
    data_name="cars"
    column="DISTINCT batchname"
    db_name = "myown.db"
    try:
        lines=getColumn(data_name, column, db_name)
    except:
        print("Error when getting row count, assuming row count to be 0")
        return []
    res = []
    for l in lines:
        #print(l[0])
        res.append(l[0])
    return res

def insertCsvToDb():
    db_name="myown.db"
    data_name="cars_raw"
    dir_name="data"
    files_inserted=getFilesInserted()
    #print(files_inserted)
    failed=0
    data_dir=os.listdir('./'+dir_name)
    data_dir.sort()
    #print(data_dir)
    for filename in data_dir:
        if filename not in files_inserted:
            date = filename.split(" ")[0]
            print("Inserting " + filename)
            results = []
            with open('./'+dir_name+"/"+filename) as csvfile:
                reader = csv.reader(csvfile) # change contents to floats
                try:
                    for row in reader: # each row is a list
                        try:
                            if len(row)>0:
                                results.append(row)
                        except:
                            failed=failed+1
                            #raise
                    insertDB(db_name, data_name, results, date, filename)
                    print(filename)
                except Exception as e:
                    print("exception "+str(e))
                    #raise
        else:
            print(filename, " already inserted")

    print("Failed "+str(failed))
    
insertCsvToDb()

In [ ]:
#Select from cars_raw and insert the select into cars

#based on join decide which to add - outer join

#See which records have latest date and make them current

#See which records have 'exited' from the data, thus making them sold -> add status field

#See which records have later records, being 'intermediate' records


In [ ]:
#analysis

#get data for histogram presentation, try logarithmic prices etc

#make the histogram based on car model, do it for top 40 cars